# Preprocesamiento de los comentarios para la primera fase de fine-tuning y entrenamiento del modelo
En esta primera fase, una vez extraídos los bodies de todos los issues/PRs de los repositorios listados junto con 2 comentarios de 250 del total de issues de cada repositorio, se prepararán todos los datos para ser utilizados en el entrenamiento y ajuste fino del modelo que se utilizará.

Se seguirán prácticamente los mismos pasos vistos en *c_preparing_data_for_statistics_and_ML* pero con varias diferencias claves que existen entre los modelos BERT que se utilizarán ahora y los modelos de clasificación presentados con anterioridad (notebooks de GVTIA).

Para Transformers funciona mejor un preprocesado mínimo y dejar la segmentación al propio tokenizador del modelo, a continuación se muestra que procedimientos similares a los anteriores se mantendrán y cuáles se evitarán:

## Se mantendrá:
- Normalización de espacios/saltos de línea
- Eliminación de caracteres de control raros o poco usuales
- Se conservará el uso de mayúsculas y minúsculas, signos, números, URLs, nombres propios de vulnerabilidades o bugs (CVE-XXXX-YYYY), rutas (/etc/...), código entre backticks (`return salida`), nombres de APIs.
- Se definirá una longitud máxima de tokens por comentario o el uso de *sliding window* si el texto es muy largo.

## Se omitirá:
- Pasar todo el texto a minúsculas, los modelos RoBERTa/DistilRoBERTa que se utilizarán utilizan mayúsculas y minúsculas.
- Eliminar la puntuación y stopwords.
- Stemming / lematización.
- Normalizaciones agresivas de URLs/código -> se pierde señal técnica.

Una vez explicado esto, se comenzará con el preprocesado de todos los comentarios extraídos de GitHub, comenzando como se ha visto ya en diversas ocasiones, con cargar el documento (.csv) en un dataframe de pandas para su uso y manipulación.

En este caso, se cuenta con 2 documentos:
- **gh_bodys_lastyear.csv**. Archivo que contiene los bodies (comentario principal) de todos los Issues/PRs en el último año de los repositorios listados para la extracción de comentarios.
- **gh_comments_lastyear.csv**. Archivo que contiene los 2 primeros comentarios de cada Issue/PR de 250 Issues/PRs por repositorio (500 comentarios por repo), en gran parte de los casos serán las respuestas aportadas por usuarios al body del documento anterior.

En este caso, como se cuenta con 2 documentos lo que se hará es crear 2 dataframes, uno con cada documento, para a continuación unirlos con la función `concat()` de pandas y ordenarlos según el id del Issue/PR para la clara visualización y mantener una estructura coherente entre cuerpo principal y comentarios asociados.

In [1]:
# === Bootstrap cachés HF y PIP en ruta del proyecto ===
import os, pathlib

PROJ = pathlib.Path.home() / "BERTolto"
HF_HOME = PROJ / ".hf_home"
PIP_CACHE = PROJ / ".cache" / "pip"

os.environ.setdefault("HF_HOME", str(HF_HOME))
os.environ.setdefault("HUGGINGFACE_HUB_CACHE", str(HF_HOME / "hub"))
os.environ.setdefault("TRANSFORMERS_CACHE", str(HF_HOME / "transformers"))
os.environ.setdefault("HF_DATASETS_CACHE", str(HF_HOME / "datasets"))
os.environ.setdefault("PIP_CACHE_DIR", str(PIP_CACHE))
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
os.environ.setdefault("HF_HUB_DISABLE_TELEMETRY", "1")
# (En entornos de "solo tokenizador", evitar intentar TF/JAX)
os.environ.setdefault("USE_TF", "0")
os.environ.setdefault("USE_JAX", "0")

for p in (HF_HOME, HF_HOME/"hub", HF_HOME/"transformers", HF_HOME/"datasets", PIP_CACHE):
    p.mkdir(parents=True, exist_ok=True)

print("HF_HOME:", os.environ["HF_HOME"])
print("TRANSFORMERS_CACHE:", os.environ["TRANSFORMERS_CACHE"])
print("HF_DATASETS_CACHE:", os.environ["HF_DATASETS_CACHE"])
print("PIP_CACHE_DIR:", os.environ["PIP_CACHE_DIR"])

HF_HOME: /home/diego/BERTolto/.hf_home
TRANSFORMERS_CACHE: /home/diego/BERTolto/.hf_home/transformers
HF_DATASETS_CACHE: /home/diego/BERTolto/.hf_home/datasets
PIP_CACHE_DIR: /home/diego/BERTolto/.cache/pip


In [2]:
import os
import subprocess
# Cargamos el archivo settings_bert.py que contiene todos los imports necesarios para la correcta ejecución del notebook y no tener que importarlos en cada celda de código
%run -i ../settings_bert.py

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
# RUTA del CSV de Reddit (tal como lo has scrapeado)
path_reddit_comments = "../data/reddit_comments/training/comentarios_reddit_raw_lastyear.csv"

# 1) Cargar
df_re = pd.read_csv(path_reddit_comments)

# 2) Normalizar nombres de columnas -> a lo que esperan nuestros scripts genéricos
#    - text      <- comment_body
#    - created_utc <- comment_created_utc   (prep_utils reconocerá created_utc)
#    - link_id   <- submission_id           (prep_utils lo usa para context_id)
#    - url       <- comment_permalink       (sirve para inferir source=reddit)
rename_map = {
    "comment_body": "text",
    "comment_created_utc": "created_utc",
    "submission_id": "link_id",
    "comment_permalink": "url",
}
df_re = df_re.rename(columns={k:v for k,v in rename_map.items() if k in df_re.columns})

# 3) (Opcional pero recomendable) sello de fuente explícito
df_re["source"] = "reddit"

# 4) Guardar CSV temporal ya “amigable”
tmp_norm_csv = "../data/reddit_comments/training/comentarios_reddit_norm.csv"
Path(tmp_norm_csv).parent.mkdir(parents=True, exist_ok=True)
df_re.to_csv(tmp_norm_csv, index=False, encoding="utf-8")

print("OK. Columnas normalizadas y CSV temporal guardado en:", tmp_norm_csv)
print("Cols:", df_re.columns.tolist()[:20])
print("Filas:", len(df_re))

OK. Columnas normalizadas y CSV temporal guardado en: ../data/reddit_comments/training/comentarios_reddit_norm.csv
Cols: ['subreddit', 'link_id', 'submission_created_utc', 'submission_title', 'comment_id', 'created_utc', 'comment_author', 'text', 'url', 'submission_url', 'source']
Filas: 31335


In [4]:
# OPCIONAL: guardar también en SQLite (tabla nueva) por trazabilidad
db_re = "../data/reddit_comments/training/reddit_dataset_lastyear.db"
con = sqlite3.connect(db_re)
df_re.to_sql('reddit_dataset', con, if_exists='replace', index=False)
con.close()
print("Reddit guardado en SQLite:", db_re, "tabla=reddit_dataset")

Reddit guardado en SQLite: ../data/reddit_comments/training/reddit_dataset_lastyear.db tabla=reddit_dataset


Ahora sí se procederá al procesamiento del dataset para dejarlo preparado para el modelo BERT que se utilizará, RoBERTa o DistilRoBERTa. Este proceso se va a definir en una serie de scripts .py, cada uno con el objetivo de realizar una tarea para su reutilización en otros puntos del proyecto (cuando se haga el de reddit, u otros comentarios de github) de forma que estos sean agnósticos al sistema del que se extraen los comentarios que serán utilizados por el modelo.

Del mismo modo, tras el procesamiento de los datos, el resultado del procesado será almacenado en archivos `.parquet` por su ligereza y agilidad a la hora de ser manipulados y consumidos por modelos BERT. Las principales ventajas de este formato son:
- Más pequeño: compresión por columna, pesa de 2 a 5 veces menos que un `.csv`
- Más rápido: lee solo las columnas que se necesitan ("column pruning") y aplica vectorización.
- Conserva tipos: fechas, booleanos, enteros "nullable", etc. (`.csv`los pierde)
- Esquema: guarda el _schema_ dentro del archivo -> menos sorpresas al cargarlo

Por estas características el formato es el preferido para pipelines de datos/ML. En este caso la estructura que se utilizará será:
- `merged.parquet` = será el dataset completo tras ingesta y normalización ligera.
- `split_train.parquet`, `split_dev.parquet`, `split_test.parquet` = particiones del dataset ya divididas, listas para su tokenización.

#### Ventajas frente a CSV
- Evita problemas de comas y saltos de línea
- Mantiene las fechas (`created_at`), booleanos (`is_pr`) y enteros sin perder el tipo.
- Carga solo las columnas necesarias -> menor uso de RAM y tiempo de ejecución.

## Ejecución del preprocesado de datos
Se han definido varios scripts, cada uno con una función en el preprocesado de datos para modelos BERT, a  los cuáles se les llamará desde este notebook con los argumentos correspondientes para realizar este proceso.

Las funciones reutilizables para el pipeline están definidas en `prep_utils.py`. Normaliza texto "ligero", mapea columnas heterogéneas (GH/Reddit) al esquema core (id, text, label, source, created_at, context_id), lee CSV/SQLite y guarda Parquet.
El objetivo principal de estas funciones es su utilización cuando se desea agnosticismo de fuente (Github/Reddit) y un preprocesado mínimo ideal para Transformers.

### `ingest_merge.py`
Combina una o varias entradas en un único DataFrame, deduplica el contenido por `id`, aplica normalización ligera y guarda `merged.parquet` + meta. Se le da como input el archivo `.csv`, produciendo en la salida un DF en formato `.parquet`.

In [5]:
base = "../src/data_prep"

# 1) Ingesta + merge (desde CSV normalizado Reddit)
subprocess.run([
    sys.executable, f"{base}/ingest_merge.py",
    "--inputs", tmp_norm_csv,                         # <<< CSV Reddit normalizado
    "--out-parquet", "../src/artifacts_rd/prep/merged_reddit.parquet",
    "--out-meta",    "../src/artifacts_rd/prep/merged_meta_reddit.json"
], check=True)

print("OK -> merged_reddit.parquet")


OK -> ../src/artifacts_rd/prep/merged_reddit.parquet
OK -> merged_reddit.parquet


### `quick_report.py`
Muestra por pantalla un resumen del Parquet aportado en el input: nº de filas, distribución de `label`, distribución de `source`, rango de `created_at`.
Es útil para la verificación visual de que la ingesta es correcta antes del split/tokenización e ideal para detectar desbalanceos fuertes o rangos temporales inesperados antes de la fase de entrenamiento.

In [6]:
subprocess.run([
    sys.executable, f"{base}/quick_report.py",
    "--in-parquet", "../src/artifacts_rd/prep/merged_reddit.parquet"
], check=True)

rows: 31335
labels: {0: 31335}
fuentes: {'reddit': 31335}
rango de fechas: 2024-10-19 18:21:04+00:00 -> 2025-10-19 10:42:38+00:00


CompletedProcess(args=['/home/diego/BERTolto/.venv/bin/python', '../src/data_prep/quick_report.py', '--in-parquet', '../src/artifacts_rd/prep/merged_reddit.parquet'], returncode=0)

### `split_thread_temporal.py`
Divide el DF `merged.parquet` en train/dev/test respetando hilos (`context_id`) y orden temporal para evitar fuga de información entre splits manteniendo el orden cronológico dentro de cada uno.
Los `ratios` son las proporciones con las que se dividirá el dataset:
- train: 70%
- dev: 15%
- test: 15%

In [7]:
subprocess.run([
    sys.executable, f"{base}/split_thread_temporal.py",
    "--in-parquet", "../src/artifacts_rd/prep/merged_reddit.parquet",
    "--out-train",  "../src/artifacts_rd/prep/split_train_reddit.parquet",
    "--out-dev",    "../src/artifacts_rd/prep/split_dev_reddit.parquet",
    "--out-test",   "../src/artifacts_rd/prep/split_test_reddit.parquet",
    "--ratios",     "0.7", "0.15", "0.15",
    "--out-meta",   "../src/artifacts_rd/prep/split_meta_reddit.json"
], check=True)

print("OK -> splits reddit")


OK -> ../src/artifacts_rd/prep/split_train_reddit.parquet ../src/artifacts_rd/prep/split_dev_reddit.parquet ../src/artifacts_rd/prep/split_test_reddit.parquet
OK -> splits reddit


### `tokenize_hf.py`
Carga los Parquet de train/dev/test, antepone un prefijo de dominio (`<GITHUB>`, `<REDDIT>`, en este caso no porque todavía estoy solo con los comentarios de GH), tokeniza los textos con el tokenizador de DistilRoBERTa/RoBERTa y guarda:
- `dataset/` (formato `save_to_disk` de HF Datasets)
- `tokenizer/` (vocabulario + config)
- `preprocess_meta.json` (tamaños, `max_len`, pesos de cada clase)

#### Parámetros
- `--base-model`: modelo que se utilizará, en este caso distilroberta
- `--max-len 384`: longitud de secuencia; bajar acelera, subir captura más contexto.
- `--use-domain-prefix`: etiqueta de la fuente (GITHUB/REDDIT)
- `--sliding-window --slide-stride 128`: para textos muy largos (menos truncado)

Para decidir el `--max-len` y la posibilidad de utilizar `--sliding-window`, se puede calcular la distribución de longitudes en tokens y elegir el valor del argumento para cubrir el *p90-p95* y obtener un mejor resultado.

In [8]:
import os
# Evita que transformers intente cargar PyTorch/TF/JAX solo para el tokenizador
os.environ["USE_TORCH"] = "0"
os.environ["USE_TF"]    = "0"
os.environ["USE_JAX"]   = "0"

from transformers import AutoTokenizer, logging

tok = AutoTokenizer.from_pretrained("distilroberta-base", use_fast=True)
df = pd.read_parquet("../src/artifacts_rd/prep/merged_reddit.parquet")
lens = df["text"].astype(str).map(lambda s: len(tok(s, truncation=False)["input_ids"]))
print(pd.Series(lens).describe(percentiles=[.5, .9, .95, .99]))

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


count   31335.00
mean       64.65
std        95.41
min         3.00
50%        36.00
90%       145.00
95%       211.00
99%       440.00
max      3412.00
Name: text, dtype: float64


### Argumentos del tokenizador
- `--max-len 384`. Como el modelo solo puede leer hasta N tokens al mismo tiempo, se limita a N tokens (384) en la tokenización de cada texto. Si este tiene un nº superior se trunca o se divide en trozos (sliding-window). Elijo 384 tokens máx. por texto por su equilibrio en velocidad y uso de VRAM (no debería de tener problema con mi PC).
- `--sliding-window`. Si un texto no cabe en `--max-len`, en lugar de descartar el sobrante se divide en ventanas solapadas, dando como resultado varias filas por cada texto largo.
- `--slide-stride 128`. Es el número de tokens que se solapan entre ventanas para no cortar frases/palabras a la mitad cuando se divide un texto entre ventanas. En este caso si el texto se divide en 2 ventanas la distribución sería:
    - Ventana 1: tokens 1-384
    - Ventana 2: tokens (384-128) 256-639
- `--max-windows-per-doc 6`. Limita a un máximo de 6 ventanas por texto, el resto se recorta. Es mejor que descartar el texto completo, y si supera el máx. de ventanas es muy probable que sea debido a mensajes de logs o diffs y no aporte nada al modelo, solo ruido.
- `--filter-max-input-tokens`. Como se ve en la celda anterior (porcentajes) hay textos enormes y con este argumento se descartan sus colas limitando a N tokens antes de dividir en ventanas.
- `--num-proc 12`. Usa 12 hilos de ejecución para map paralelo (Mi PC tiene 8C/16T) -> Implementar en el tokenizador
- `--batch-chunk-size 512`. Indica el tamaño de trozos (grandes) en cada llamada al tokenizador. -> Implementar en el tokenizador

#### Recomendaciones
- Si ves que sube mucho el nº de ejemplos → baja --max-windows-per-doc a 4.
- Si va justo de RAM → --num-proc 8 y/o --batch-chunk-size 256.

In [10]:
from pathlib import Path
import os, sys, subprocess

base = "../src/data_prep"  # si ya lo tienes definido arriba, omite esta línea

# ---- Cache HF local, compartido con contenedor ----
HF_CACHE = "/home/diego/BERTolto/.cache/huggingface"
Path(HF_CACHE).mkdir(parents=True, exist_ok=True)
Path(f"{HF_CACHE}/hub").mkdir(parents=True, exist_ok=True)

env = os.environ.copy()
env["PYTHONIOENCODING"] = "utf-8"
env["PYTHONUTF8"] = "1"
env["TOKENIZERS_PARALLELISM"] = "false"
env["RAYON_NUM_THREADS"] = "2"

# caches HF para evitar ~/.cache/huggingface (que puede estar a nombre de root)
env["HF_HOME"] = HF_CACHE
env["HUGGINGFACE_HUB_CACHE"] = HF_CACHE
env["TRANSFORMERS_CACHE"] = f"{HF_CACHE}/hub"

# ---- Sanity check de entradas ----
inputs = [
    "../src/artifacts_rd/prep/split_train_reddit.parquet",
    "../src/artifacts_rd/prep/split_dev_reddit.parquet",
    "../src/artifacts_rd/prep/split_test_reddit.parquet",
]
for p in inputs:
    if not Path(p).exists():
        raise FileNotFoundError(f"Falta el archivo: {p}")

# ---- Comando tokenización (ojo al flag con 'windows' en plural) ----
cmd = [
    sys.executable, f"{base}/tokenize_hf.py",
    "--train-parquet", inputs[0],
    "--dev-parquet", inputs[1],
    "--test-parquet", inputs[2],
    "--out-dir", "../src/artifacts_rd/hf_distilroberta",
    "--base-model", "distilroberta-base",
    "--max-len", "384",
    "--sliding-window",
    "--slide-stride", "128",
    "--max-windows-per-doc", "8",  # <- nombre de flag corregido
    "--filter-max-input-tokens", "8192",
    # "--use-domain-prefix",                # actívalo cuando mezcles fuentes
    "--num-proc", "8",
    "--batch-chunk-size", "512",
]

# ---- Ejecuta mostrando logs aunque haya error ----
res = subprocess.run(cmd, env=env, capture_output=True, text=True, encoding="utf-8", errors="replace")
print("=== STDOUT ===\n", res.stdout)
print("=== STDERR ===\n", res.stderr)
res.check_returncode()  # si hubo error, aquí explota, pero ya viste los logs

=== STDOUT ===
 OK -> ../src/artifacts_rd/hf_distilroberta/dataset ../src/artifacts_rd/hf_distilroberta/tokenizer

=== STDERR ===
 /home/diego/BERTolto/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (476 > 384). Running this sequence through the model will result in indexing errors

Map (num_proc=8): 100%|██████████| 16225/16225 [00:01<00:00, 13400.80 examples/s]

Map (num_proc=8): 100%|██████████| 8303/8303 [00:01<00:00, 7815.52 examples/s] 

Map (num_proc=8): 100%|██████████| 6807/6807 [00:01<00:00, 6460.65 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 16640/1

**Factor de expansión** -> Si se va demasiado, bajar --max-window a 4

In [12]:
from datasets import load_from_disk

BASE_ARTI_R = Path("../src/artifacts_rd/hf_distilroberta")
ds_r  = load_from_disk(str(BASE_ARTI_R / "dataset"))
tok_r = AutoTokenizer.from_pretrained(str(BASE_ARTI_R / "tokenizer"), use_fast=True)

print({k: len(ds_r[k]) for k in ds_r.keys()})
print("Ejemplo decodificado:", tok_r.decode(ds_r["train"][0]["input_ids"])[:300])


{'train': 16640, 'validation': 8421, 'test': 6917}
Ejemplo decodificado: <s>Awesome</s>


**Sanity check** -> Comprobar que no hay basura

In [13]:
tokenizer.decode(tok_train["input_ids"][0])[:300]

NameError: name 'tokenizer' is not defined

**Velocidad** -> Si el tiempo de ejecución es muy elevado usar --max-len 256 o subir --slide-stride a 192 para menos ventanas